In [1]:
%matplotlib inline
import os

import cftime

import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cmocean

import cartopy
import cartopy.crs as ccrs

import esmlab

import intake
import intake_esm

In [2]:
cesm2 = intake.open_esm_datastore('/glade/work/mlevy/CMIP6-CESM2_only-NOT_CMORIZED.json')
cesm2.df.head()

,experiment,case,component,stream,variable,date_range,member_id,path,year_offset
0,esm-piControl,b.e21.B1850.f09_g17.CMIP6-esm-piControl.001,atm,cam.h1,ACTNL,00010101-00101231,0,/glade/collections/cdg/timeseries-cmip6/b.e21....,NaN
1,esm-piControl,b.e21.B1850.f09_g17.CMIP6-esm-piControl.001,atm,cam.h1,ACTNL,00110101-00201231,0,/glade/collections/cdg/timeseries-cmip6/b.e21....,NaN
2,esm-piControl,b.e21.B1850.f09_g17.CMIP6-esm-piControl.001,atm,cam.h1,ACTNL,00210101-00301231,0,/glade/collections/cdg/timeseries-cmip6/b.e21....,NaN
3,esm-piControl,b.e21.B1850.f09_g17.CMIP6-esm-piControl.001,atm,cam.h1,ACTNL,00310101-00401231,0,/glade/collections/cdg/timeseries-cmip6/b.e21....,NaN
4,esm-piControl,b.e21.B1850.f09_g17.CMIP6-esm-piControl.001,atm,cam.h1,ACTNL,00410101-00501231,0,/glade/collections/cdg/timeseries-cmip6/b.e21....,NaN


In [3]:
cesm2.df.columns

Index(['experiment', 'case', 'component', 'stream', 'variable', 'date_range',
       'member_id', 'path', 'year_offset'],
      dtype='object')

In [ ]:
cmip5 = intake.open_esm_datastore('/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip5.json')
cesm1 = cmip5.search(model='CESM1-BGC')
cesm1.df.head()

In [4]:
keep_vars = ['TAREA', 'TLONG', 'TLAT', 'IRON_FLUX', 'time', 'time_bound', 'member_id']

In [5]:
# WORKAROUND FOR intake-esm ISSUE: limit to members with only a single file

cat = cesm2.search(experiment=['historical'], variable='IRON_FLUX')
cat.df.set_index(['member_id', 'date_range'])['path']

member_id  date_range   
1          185001-201412    /glade/collections/cdg/timeseries-cmip6/b.e21....
2          185001-201412    /glade/collections/cdg/timeseries-cmip6/b.e21....
3          185001-201412    /glade/collections/cdg/timeseries-cmip6/b.e21....
4          185001-201412    /glade/collections/cdg/timeseries-cmip6/b.e21....
5          185001-201412    /glade/collections/cdg/timeseries-cmip6/b.e21....
6          185001-201412    /glade/collections/cdg/timeseries-cmip6/b.e21....
7          185001-189912    /glade/collections/cdg/timeseries-cmip6/b.e21....
           190001-194912    /glade/collections/cdg/timeseries-cmip6/b.e21....
           195001-199912    /glade/collections/cdg/timeseries-cmip6/b.e21....
           200001-201412    /glade/collections/cdg/timeseries-cmip6/b.e21....
8          185001-189912    /glade/collections/cdg/timeseries-cmip6/b.e21....
           190001-194912    /glade/collections/cdg/timeseries-cmip6/b.e21....
           195001-199912    /glade/coll

In [6]:
# TODO: eventually we should be able to remove the member_id from the search
#dq = cesm2.search(experiment=['historical'], variable='IRON_FLUX', member_id=[1,2,3,4,5,6]).to_dataset_dict(cdf_kwargs={'chunks':{'time': 48}, 'decode_times' : False})
#dq = cesm2.search(experiment=['historical'], variable='IRON_FLUX', member_id=[1,2,3,4,5,6]).to_dataset_dict(cdf_kwargs={'decode_times' : False})
dq = cesm2.search(experiment=['historical'], variable='IRON_FLUX', member_id=[1,2,3,4,5,6]).to_dataset_dict(cdf_kwargs={'chunks':{'time': 48}})
#dq = cesm2.search(experiment=['historical'], variable='IRON_FLUX', member_id=[1,2,3,4,5,6]).to_dataset_dict()

_, ds2 = dq.popitem()
ds2 = ds2.drop([v for v in ds2.variables if v not in keep_vars])
ds2

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 1 group(s)


<xarray.Dataset>
Dimensions:     (d2: 2, member_id: 6, nlat: 384, nlon: 320, time: 1980)
Coordinates:
  * member_id   (member_id) int64 1 2 3 4 5 6
  * time        (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
Dimensions without coordinates: d2, nlat, nlon
Data variables:
    TAREA       (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLONG       (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLAT        (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    time_bound  (time, d2) object dask.array<chunksize=(48, 2), meta=np.ndarray>
    IRON_FLUX   (member_id, time, nlat, nlon) float32 dask.array<chunksize=(1, 48, 384, 320), meta=np.ndarray>
Attributes:
    revision:          $Id: tavg.F90 89644 2018-08-04 14:26:01Z klindsay $
    calendar:          All years have exactly  365 days.
    history:           none
    contents:          Diagnostic and Prognostic Variables
    time_period_freq:  month_1


In [ ]:
dq = cesm1.search(experiment='historical', variable='IRON_FLUX').to_xarray(chunks={'time': 48})
_, ds1 = dq.popitem()
ds1 = ds1.drop([v for v in ds1.variables if v not in keep_vars])
ds1

In [7]:
ds2_ann = esmlab.resample(ds2, freq='ann')
ds2_ann

Please open dataset with `decode_times=False`


AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
ds1_ann = esmlab.resample(ds1, freq='ann')
ds1_ann

In [ ]:
iron_flux_glb2 = esmlab.weighted_sum(ds2_ann.IRON_FLUX, dim=['nlat', 'nlon'], weights=ds2_ann.TAREA)
iron_flux_glb2 = iron_flux_glb2 * 1.0e-16 * 365.0 * 86400.0 # mmolFe/m^2/s to GmolFe/y
iron_flux_glb2.attrs['units'] = 'Gmol Fe yr$^{-1}$'
iron_flux_glb2 = iron_flux_glb2.compute()
iron_flux_glb2

In [ ]:
iron_flux_glb1 = esmlab.weighted_sum(ds1_ann.IRON_FLUX, dim=['nlat', 'nlon'], weights=ds1_ann.TAREA)
iron_flux_glb1 = iron_flux_glb1 * 1.0e-16 * 365.0 * 86400.0 # mmolFe/m^2/s to GmolFe/y
iron_flux_glb1.attrs['units'] = 'Gmol Fe yr$^{-1}$'
iron_flux_glb1 = iron_flux_glb1.compute()
iron_flux_glb1

In [ ]:
plot_name = f'timeseries.global.IRON_FLUX.pdf'

fig = plt.figure()
ax = fig.add_subplot(111)
for i in iron_flux_glb2.member_id.values:
    ax.plot(iron_flux_glb2.time, iron_flux_glb2.sel(member_id=i), linewidth=0.5)

ax.plot(iron_flux_glb2.time, iron_flux_glb2.mean('member_id'), color='k', linewidth=1.5)    

ax.plot(iron_flux_glb1.time, iron_flux_glb1, linestyle='--', color='k', linewidth=1.5)    

ax.set_ylabel('Iron deposition [Gmol Fe yr$^{-1}$]');

ax.text(cftime.DatetimeNoLeap(1990, 1, 1), iron_flux_glb2.sel(time='1990').max('member_id')*1.05, 'CESM2',color='k');

ax.text(cftime.DatetimeNoLeap(1965, 1, 1), iron_flux_glb1.sel(time='1960')*1.02, 'CESM1',color='k');

#plt.savefig(plot_name,dpi=300,bbox_inches='tight')

In [ ]:
mmolm2s_to_molm2yr = 1e-3 * 86400. * 365.
iron_flux_map2 = ds2_ann.IRON_FLUX.sel(time=slice('1995', '2014')).mean(['time', 'member_id'])
iron_flux_map2 = iron_flux_map2.compute() * mmolm2s_to_molm2yr
iron_flux_map2

In [ ]:
iron_flux_map2

In [ ]:
fig = plt.figure()
ax = plt.subplot(1, 1, 1, projection=ccrs.Robinson(central_longitude=305.0))


pc = ax.pcolormesh(ds2.TLONG, ds2.TLAT, iron_flux_map2,
                   norm=colors.LogNorm(vmin=1e-6, vmax=1e-3),
                   cmap=cmocean.cm.matter,
                   transform=ccrs.PlateCarree())

ax.add_feature(cartopy.feature.NaturalEarthFeature('physical','land','110m',
                                                   edgecolor='face',
                                                   facecolor='lightgray'))
ax.set_global() 
ax.coastlines(linewidth=0.5)
cb = plt.colorbar(pc, shrink=0.6)
cb.ax.set_title('mol m$^{-2}$ yr$^{-1}$');

In [ ]:
iron_flux_map2.max()

In [ ]:
cesm2.search(experiment=['historical', 'SSP5-8.5'], variable='IRON_FLUX').to_xarray(chunks={'time': 48})